In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-te/

/content/nmt-kn-te


In [3]:
! chmod a+x /content/nmt-kn-te/scripts/data_preparation.sh

In [4]:
! chmod a+x /content/nmt-kn-te/scripts/split_dataset.sh

In [5]:
! /content/nmt-kn-te/scripts/data_preparation.sh data.kn data.te

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 13.95 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 3.26 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 16.90 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads

In [6]:
! /content/nmt-kn-te/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.te

In [7]:
%cd ..

/content


In [8]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [9]:
!fairseq-preprocess --source-lang kn --target-lang te \
--trainpref nmt-kn-te/train/train \
--validpref nmt-kn-te/dev/dev \
--testpref nmt-kn-te/test/test \
--destdir nmt-kn-te/tokenized.kn-te \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-06 03:32:33 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 03:32:33 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-te/tokenized.kn-te', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [10]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
!fairseq-train /content/nmt-kn-te/tokenized.kn-te \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "BiLSTM model - kn to te - 6 March 2022.ipynb"

2022-03-06 03:34:49 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-06 03:34:50 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 03:34:53 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'BiLSTM model - kn to te - 6 March 2022.ipynb', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_si

In [ ]:
!pwd

In [12]:
!fairseq-generate /content/nmt-kn-te/tokenized.kn-te \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-919	-0.6979048848152161	ప ్ రత ్ యామ ్ నాయ మార ్ గం
P-919	-2.0775 -0.0007 -1.9730 -0.0006 -1.0371 -0.0014 -1.3449 -0.2734 -0.0010 -0.3730 -0.5943
S-731	ಯಾರೂ ಬರೆಯುತ ್ ತಾರೆ ನಾನು ಬರೆಯುತ ್ ತೇನೆ ಇದು ಬೇಡ .
T-731	నాకు ఎవరూ రాయలేదు కూడా .
H-731	-1.5208156108856201	నేను ఎవ ్ వరూ కాదనకూడదు .
D-731	-1.5208156108856201	నేను ఎవ ్ వరూ కాదనకూడదు .
P-731	-2.4580 -4.3671 -0.0605 -0.1124 -2.7175 -2.4017 -0.0355 -0.0138
S-776	&quot; &quot; &quot; ಕಮ ್ ಆನ ್ ಡೌನ ್ ! &quot;
T-776	“ లోపలికి రావచ ్ చా !
H-776	-1.226524829864502	&apos; &apos; ఫ ్ రాన ్ స ్ ‌ !
D-776	-1.226524829864502	&apos; &apos; ఫ ్ రాన ్ స ్ ‌ !
P-776	-2.0497 -0.7445 -4.7700 -0.1130 -2.2748 -0.0004 -0.3245 -0.0041 -0.7586 -2.4258 -0.0265
S-852	ಸ ್ ವಲ ್ ಪ ಭಯವೂ ಆಗುತ ್ ತಿದೆ .
T-852	కాస ్ త భయం పోతుంది .
H-852	-1.1826815605163574	ఇది వేసవి రోజులు గడిచాయి .
D-852	-1.1826815605163574	ఇది వేసవి రోజులు గడిచాయి .
P-852	-3.3933 -2.7580 -0.5058 -1.4941 -0.0938 -0.0320 -0.0018
S-91	ಆಕೆಯ ಮಾತುಗಳು ಸ ್ 